<a href="https://colab.research.google.com/github/ryukya/AI_2020/blob/SVM/Half_Features_Mushroom_SVM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data preparation

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

##load data
df_mushroom = pd.read_csv('mushrooms.csv')

##transform value
lb=LabelEncoder()
df_mushroom=df_mushroom.apply(lb.fit_transform)
datas=df_mushroom.rename(columns={ "class": "lab"}, errors="raise")

datas.head()


,lab,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


In [2]:
##separate label and features data
dt_label = datas['lab']
df_mushroom = datas.drop(['lab'],axis=1)
##put data to Y 
Y = []
for val in dt_label:
    if(val == 1): ## val=1 poisonus
        Y.append(-1)
    else:
        Y.append(1)
print(df_mushroom.head())


   cap-shape  cap-surface  cap-color  ...  spore-print-color  population  habitat
0          5            2          4  ...                  2           3        5
1          5            2          9  ...                  3           2        1
2          0            2          8  ...                  3           2        3
3          5            3          8  ...                  2           3        5
4          5            2          3  ...                  3           0        1

[5 rows x 22 columns]


##Removing Random Features ( 50% of Total Features)

In [3]:
from random import seed
from random import randint

seed(1)
cols = []
col_tot= len(df_mushroom.columns)
half=int(col_tot*0.5)
for _ in range(half):
	value = randint(0, col_tot)
	cols.append(value)
print (cols)

df_mushroom.drop(df_mushroom.columns[cols],axis=1,inplace=True)
df_mushroom.shape

[4, 18, 2, 8, 3, 15, 14, 15, 20, 12, 6]


(8124, 12)

In [4]:
##put data to X
X = df_mushroom.values.tolist()

## prepare variable
x_train = []
y_train = []
x_test = []
y_test = []

##split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=10)

##data leng for data training and testing
lengt=len(df_mushroom)
train_len= int(0.8*lengt)
test_len=int( 0.2*lengt)+1

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

y_train = y_train.reshape(train_len,1)
y_test = y_test.reshape(test_len,1)
print("length: ",lengt,"---- training:",train_len,"----test:",test_len)



length:  8124 ---- training: 6499 ----test: 1625




##Evaluation Function

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
def evaluate (y_test,predictions):
  print ("Accuracy Score")
  print(accuracy_score(y_test,predictions))
  print ("Confusion Matrix")
  print(confusion_matrix(y_test, predictions))
  print ("Classification report")
  print(classification_report(y_test, predictions))

##SVM Function

In [0]:
def svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc):
  training=[]
  ##train_fx & training reshape 
  for col in range(x_train.shape[1]):
    training.append(x_train[:,col].reshape(train_len,1))
  ##make w array (zeros) 
  w_zeros=[]
  for x in training:
    w_zeros.append(np.zeros((train_len,1)))
  epochs = 1
  alpha = alpa
  while(epochs < epoc):
    ##sum of array in w_zeros*array in training
    sumval = None 
    for xw,yt in zip(w_zeros, training):
      xy=xw*yt
      if sumval is None:
       y1=xy
      else:
        y1= y1 + xy
      sumval =y1 
    prod = sumval * y_train
    count = 0
    for val in prod:
      if(val >= 1):
        cost = 0
        for i in range(len(w_zeros)):
         w_zeros[i]= w_zeros[i] - alpha * (2 * 1/epochs * w_zeros[i])
      else:
        cost = 1 - val
        for i in range(len(w_zeros)):
         w_zeros[i]= w_zeros[i] + alpha * (training[i][count] * y_train[count] - 2 * 1/epochs * w_zeros[i])
      count += 1
    epochs += 1
  ## Clipping weights  and reshape array
  index = list(range(test_len,train_len))
  for i in range(len(w_zeros)):
    w_zeros[i]=np.delete(w_zeros[i],index)
    w_zeros[i] = w_zeros[i].reshape(test_len,1)
  ## Extract the test data features 
  testing=[]
  for col in range(x_test.shape[1]):
    testing.append(x_test[:,col].reshape(test_len,1))
  ## Predict
  pred_val = None 
  for xw,yt in zip(w_zeros, testing):
    xy=xw*yt
    if pred_val is None:
      y1=xy
    else:
      y1= y1 + xy
    pred_val =y1 
  #add prediction result into list
  predictions = []
  for val in pred_val:
      if(val > 1):
          predictions.append(1)
      else:
          predictions.append(-1)
  evaluate(y_test,predictions)     



##SVM  with alpha= 0.002 and epoch =500

In [7]:
alpa=0.002
epoc=500
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.5316923076923077
Confusion Matrix
[[ 20 744]
 [ 17 844]]
Classification report
              precision    recall  f1-score   support

          -1       0.54      0.03      0.05       764
           1       0.53      0.98      0.69       861

    accuracy                           0.53      1625
   macro avg       0.54      0.50      0.37      1625
weighted avg       0.54      0.53      0.39      1625



##SVM  with alpha= 0.001 and epoch =1000

In [8]:
alpa=0.001
epoc=1000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.5384615384615384
Confusion Matrix
[[ 31 733]
 [ 17 844]]
Classification report
              precision    recall  f1-score   support

          -1       0.65      0.04      0.08       764
           1       0.54      0.98      0.69       861

    accuracy                           0.54      1625
   macro avg       0.59      0.51      0.38      1625
weighted avg       0.59      0.54      0.40      1625



In [9]:
alpa=0.001
epoc=1000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.5384615384615384
Confusion Matrix
[[ 31 733]
 [ 17 844]]
Classification report
              precision    recall  f1-score   support

          -1       0.65      0.04      0.08       764
           1       0.54      0.98      0.69       861

    accuracy                           0.54      1625
   macro avg       0.59      0.51      0.38      1625
weighted avg       0.59      0.54      0.40      1625



##SVM  with alpha= 0.0005 and epoch =2000

In [10]:
alpa=0.0005
epoc=2000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.7175384615384616
Confusion Matrix
[[337 427]
 [ 32 829]]
Classification report
              precision    recall  f1-score   support

          -1       0.91      0.44      0.59       764
           1       0.66      0.96      0.78       861

    accuracy                           0.72      1625
   macro avg       0.79      0.70      0.69      1625
weighted avg       0.78      0.72      0.69      1625



##SVM  with alpha= 0.0002 and epoch =5000

In [11]:
alpa=0.0002
epoc=5000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.9618461538461538
Confusion Matrix
[[723  41]
 [ 21 840]]
Classification report
              precision    recall  f1-score   support

          -1       0.97      0.95      0.96       764
           1       0.95      0.98      0.96       861

    accuracy                           0.96      1625
   macro avg       0.96      0.96      0.96      1625
weighted avg       0.96      0.96      0.96      1625



##SVM  with alpha= 0.0001 and epoch =10000

In [12]:
alpa=0.0001
epoc=10000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.955076923076923
Confusion Matrix
[[742  22]
 [ 51 810]]
Classification report
              precision    recall  f1-score   support

          -1       0.94      0.97      0.95       764
           1       0.97      0.94      0.96       861

    accuracy                           0.96      1625
   macro avg       0.95      0.96      0.95      1625
weighted avg       0.96      0.96      0.96      1625



##Result if Using Scikit SVM Library

In [13]:
from sklearn import svm
sv_f = svm.SVC(kernel='linear') # Linear Kernel
sv_f.fit(x_train, y_train)
y_pred = sv_f.predict(x_test)
evaluate(y_test, y_pred)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy Score
0.9403076923076923
Confusion Matrix
[[720  44]
 [ 53 808]]
Classification report
              precision    recall  f1-score   support

          -1       0.93      0.94      0.94       764
           1       0.95      0.94      0.94       861

    accuracy                           0.94      1625
   macro avg       0.94      0.94      0.94      1625
weighted avg       0.94      0.94      0.94      1625

